In [310]:
import numpy as np
import time
from scipy import linalg
import einops

# Maxvol

In [271]:
def Maxvol(A, eps = 1.06, iter=10):
    n, r = A.shape
    for _ in range(iter):
        i = A.shape[1]
        C = A[:i, :]
        B = A @ np.linalg.inv(C)
        index = np.unravel_index(np.argmax(B), B.shape)
        if np.round(np.abs(B[index]), 10) == 1:
            return C
        if np.round(np.abs(B[index]), 10) > 1:
            A[index, :] = A[[index[1], index[0]], :]
    return C

In [272]:
r = 3
n = 10
A = np.zeros([n, r])
for i in range(n):
    for j in range(r):
        A[i,j]= np.cos(2*i - j)
Maxvol(A)

array([[ 1.        ,  0.54030231, -0.41614684],
       [-0.41614684,  0.54030231,  1.        ],
       [-0.65364362, -0.9899925 , -0.41614684]])

# Cross

In [273]:
def Cross(A, eps = 0.1):
    r = 0
    m, n = A.shape
    I = np.arange(m)
    J = np.arange(n)
    i,j = np.unravel_index(np.argmax(np.abs(A), axis=None), A.shape)
    V = np.copy(A[:, j] / A[i, j]).reshape(m, 1)
    U = np.copy(A[i, :]).reshape(1, n)
    Error = A - V @ U
    I[i] = -1
    J[j] = -1
    i = 0
    j = 0
    while True:
        r += 1
        for k in range(n):
            if (abs(Error[k,j]) > abs(Error[i,j])) and (I[k] != -1):
                i = k
            if (abs(Error[i,k]) > abs(Error[i,j])) and (J[k] != -1):
                j = k
        if ((abs(Error[i][j]) *((m-r)*(n-r))**0.5) <= eps*(np.linalg.norm(V @ U, ord='fro'))):
            r -= 1
            return V, U
        U = np.concatenate((U, Error[i, :].reshape(1, n)), axis = 0)
        V = np.concatenate((V, (Error[:, j] / Error[i,j]).reshape(m, 1)), axis = 1)
        I[i] = -1
        J[j] = -1
        Error = A - V @ U

In [274]:
n = 16
m = 64
A = np.zeros([m, n])
for i in range(m):
    for j in range(n):
        A[i,j] = i*2 - j

In [275]:
U, V = Cross(A, 0.01)
np.linalg.norm((A - U @ V), ord = 'fro')

1.3916028932457436e-13

# Randomized SVD

In [276]:
def Randomized_SVD(A, r, p = 8):
    M, N = A.shape
    B = A @ (np.random.normal(size = (N, r + p)))
    Q, R = np.linalg.qr(B)
    U, s, V = np.linalg.svd(Q.T @ A);
    return (Q @ U, s, V)

def A(i, j):
    return i * j

M = 1024
N = 2048
r = 4
U, E, V = Randomized_SVD(np.fromfunction(A, (M, N)), r)

In [277]:
U

array([[-1.46804713e-16, -1.81858849e-01,  1.40127702e-01, ...,
        -1.56839635e-01,  4.99615663e-03,  5.16090553e-02],
       [-5.28967401e-05, -5.91802845e-04, -1.56869535e-04, ...,
        -4.66184195e-03, -2.54791517e-03,  1.99128955e-04],
       [-1.05793480e-04, -7.28668621e-04,  2.74594329e-04, ...,
        -2.13631588e-02, -1.14057001e-02, -1.07482306e-04],
       ...,
       [-5.40075717e-02,  2.99117000e-02, -1.02843795e-01, ...,
         1.22837998e-03, -2.73999607e-02,  3.07905296e-02],
       [-5.40604684e-02,  1.14609608e-02, -1.09061374e-01, ...,
         3.19634501e-02, -4.24773281e-03, -5.68777471e-02],
       [-5.41133652e-02, -4.78327723e-02,  7.55789331e-02, ...,
         3.91940322e-03,  5.20725717e-03, -3.00138221e-02]])

In [278]:
E

array([1.01122155e+09, 3.31024238e-08, 1.87406879e-08, 1.54153519e-08,
       1.47330698e-08, 1.31566642e-08, 1.26255367e-08, 1.25478473e-08,
       1.21735563e-08, 1.15367917e-08, 1.07875892e-08, 9.25024691e-09])

In [279]:
V

array([[ 0.00000000e+00, -1.86949701e-05, -3.73899402e-05, ...,
        -3.82312138e-02, -3.82499088e-02, -3.82686038e-02],
       [ 8.19636100e-16,  2.07320970e-05,  4.14641940e-05, ...,
        -2.37832304e-02,  1.25542641e-02, -7.19353659e-02],
       [-5.63931207e-16, -3.21456329e-05, -6.42912658e-05, ...,
        -1.94030926e-02,  5.62482314e-02,  1.83871144e-02],
       ...,
       [-3.82568481e-02,  8.30936634e-03, -8.01367858e-03, ...,
         9.86536077e-01,  2.08228815e-03, -3.62843995e-03],
       [-3.82607192e-02,  1.61451802e-02, -2.22342007e-03, ...,
         2.07921724e-03,  9.84831321e-01, -1.23677844e-03],
       [-3.82639374e-02,  6.58251888e-02, -1.61893886e-02, ...,
        -3.62590370e-03, -1.23673121e-03,  9.82024966e-01]])

# HO_SVD

### 3-мерный HO_SVD

In [280]:
def HO_SVD_3d(A, r1, r2, r3):
    n1,n2,n3 = A.shape
    A1 = A.reshape(n1, n2*n3)
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V1 = U[:,:r1] 
    V1 = V1.T
    A1 = np.diag(s[:r1]) @ V[:r1,:] # r1 x n2*n3
    A1 = A1.reshape(r1, n2, n3)
    A1 = A1.transpose([0,2,1])
    A1 = A1.reshape(r1 * n3, n2)# r1 * n3 x n2
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    V2 =  V[:r2,:] # r2 x n2
    U2 = U[:,:r2] @ np.diag(s[:r2]) # r1*n3 x r2
    U2 = U2.reshape(r1, n3, r2) # r1 x n3 x r2
    U2 = U2.transpose([0,2,1]) # r1 x r2 x n3
    U2 = U2.reshape(r1 * r2, n3) # r1*r2 x n3
    U, s, V = np.linalg.svd(U2, full_matrices=False)
    V3 = V[:r3,:] # U3(r3,j)
    G = U[:,:r3] @ np.diag(s[:r3])# r1 * r2 x r3
    G = G.reshape(r1, r2, r3) # main core
    return G, V1, V2, V3

def Check(A, G, V1, V2, V3): 
    err = 0
    r1, r2, r3 = G.shape
    A1 = G.reshape(r1 * r2, r3).dot(V3) # r1*r2 x n3
    A1 = A1.reshape(r1, r2, V3.shape[1]).transpose([0,2,1]) # r1 x r2 x n3
    A1 = A1.reshape(r1*V3.shape[1], r2).dot(V2) # r1*n3 x n2
    A1 = A1.reshape(r1, V3.shape[1], V2.shape[1]).transpose([1,2,0]) # n3 x n2 x r1
    A1 = A1.reshape(V3.shape[1], V2.shape[1], r1).dot(V1) # n3 x n2 x n1
    A1 = A1.transpose([2,1,0]) # n1 x n2 x n3
    return np.linalg.norm(A - A1) #error norm

In [281]:
%%time

N = 64
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
G, V1, V2, V3 = HO_SVD_3d(A,4,4,4)
print("Error =", Check(A, G, V1,V2,V3))

Error = 1.9423889953547366e-12
CPU times: user 418 ms, sys: 39.6 ms, total: 458 ms
Wall time: 372 ms


# st_HO_SVD

In [303]:
def st_HO_SVD(A: np.ndarray, eps: float = 1e-2):
    U = []
    G = A.copy()
    shape = list(A.shape)
    for k in range(A.ndim):
        Ak = np.moveaxis(G, k, 0).reshape(shape[k], -1)
        u, s, v = np.linalg.svd(Ak, full_matrices=False)
        r = s.size - np.argwhere(np.cumsum(s[::-1] ** 2) / s[0] > eps)[0][0]
        shape[k] = r

        U.append(u[:, :r])
        G = np.diag(s[:r]) @ v[:r, :]
        G = G.reshape(r, *[shape[i] for i in range(len(shape)) if i != k])
    return G.T, U

In [304]:
A = np.array([[[1, 2], [3, 4]], [[5, 6], [7, 8]]])
st_hosvd(A)

(array([[[-1.42253953e+01, -8.28025332e-03],
         [-4.61793060e-03,  1.11585148e+00]],
 
        [[-1.60125603e-02,  2.38589095e-01],
         [ 5.43770692e-01, -2.00114739e-01]]]),
 array([[-0.37616823,  0.92655138],
        [-0.92655138, -0.37616823]]),
 array([[ 0.56672424, -0.82390754],
        [ 0.82390754,  0.56672424]]),
 array([[ 0.64142303, -0.7671874 ],
        [ 0.7671874 ,  0.64142303]]))

In [305]:
def Tensor_from_Tucker(G: np.ndarray, U: list):
    fold_part = ""
    common_part = ""
    for i in range(G.ndim):
        if i > 0:
            fold_part += ","
        fold_part += chr(97 + i) + chr(65 + i)
        common_part += chr(65 + i)

    return np.einsum(common_part + "," + fold_part, G, *U)

def squeeze(G: np.ndarray, U: list, eps: float = 1e-15):
    Rs = []
    for i in range(G.ndim):
        Q, R = np.linalg.qr(U[i])
        Rs.append(R)
        U[i] = Q
    G, Un = st_HO_SVD(Tensor_from_Tucker(G, Rs), eps)
    for i in range(G.ndim):
        Rs[i] = U[i] @ Un[i]

    return G, Rs

In [306]:
G, U = st_HO_SVD(Tensor, eps=1e-13)
squeeze(G, U)

(array([[[[-3.55509865],
          [-1.22252192]],
 
         [[-1.24529779],
          [ 2.60491672]],
 
         [[ 0.14121908],
          [-0.1577141 ]]],
 
 
        [[[ 1.62448986],
          [-0.46037123]],
 
         [[-0.22713869],
          [ 1.89760818]],
 
         [[ 0.28773942],
          [ 0.34361607]]]]),
 [array([[-0.62982797, -0.77673465],
         [-0.77673465,  0.62982797]]),
  array([[-0.61526264, -0.08084034, -0.78416626],
         [-0.64534466,  0.62294301,  0.44212248],
         [-0.45274956, -0.77807895,  0.43544344]]),
  array([[-0.99984838, -0.01741326],
         [-0.01741326,  0.99984838]]),
  array([[1.]])])

# TT_SVD

In [312]:
def TT_SVD(A, ranks):
    g = []
    n = list(A.shape)
    dim = len(n)

    A1 = A.reshape(n[0], np.prod(n[1:]))
    U, s, V = np.linalg.svd(A1, full_matrices=False)
    G1 = U[:,:ranks[0]]
    g.append(G1)
    B = np.diag(s[:ranks[0]]) @ V[:ranks[0], :]
    for k in range (1, dim - 1):
        B = B.reshape(ranks[k - 1] * n[k], np.prod(n[k+1:]))
        U, s, V = np.linalg.svd(B, full_matrices=False)
        Gk = U[:,:ranks[k]]
        Gk = Gk.reshape(ranks[k - 1], n[k], ranks[k])
        g.append(Gk)
        B = np.diag(s[:ranks[k]]) @ V[:ranks[k],:]
    g.append(B)
    return g

def Restore_tensor(g):
    n = [g[i].shape[1] for i in range(len(g))]
    n[0] = g[0].shape[0]
    dim = len(n)
    
    res = g[-1]
    for i in range(dim - 2, 0, -1):
        res = (g[i] @ res).reshape(g[i].shape[0], np.prod(n[i:dim]))    
    res = g[0] @ res
    res = res.reshape(n)
    return res

In [318]:
%%time

N = 32
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
            
g = TT_SVD(A, [4, 4])
B = Restore_tensor(g)

err = np.linalg.norm(A - B)
print("error = ", err)

error =  8.901142497836701e-13
CPU times: user 94.7 ms, sys: 15.3 ms, total: 110 ms
Wall time: 81.5 ms


In [330]:
def TT_squeeze(G, eps=1e-15):
    G[0], R = np.linalg.qr(G[0])

    for i in range(1, len(G) - 1):
        G[i] = np.einsum("mk,krt->mrt", R, G[i])
        Q, R = np.linalg.qr(G[i].reshape(-1, G[i].shape[2]))
        G[i] = Q.reshape(G[i].shape[0], G[i].shape[1], -1)
    G[-1] = np.einsum("mk,kr->mr", R, G[-1])
    U, S, G[-1] = np.linalg.svd(G[-1])
    rank = S.size - np.argmax(np.cumsum(S[::-1] ** 2) / S[0] > eps)
    G[-1] = G[-1][:rank, :]
    U = U[:, :rank] * S[np.newaxis, :rank]
    for i in range(len(G) - 2, 0, -1):
        G[i] = np.einsum("mrt,tk->mrk", G[i], U)
        n = G[i].shape[1]
        rank_tmp = G[i].shape[2]
        G[i] = np.reshape(G[i], (-1, n * rank))
        U, S, G[i] = np.linalg.svd(G[i])
        rank = S.size - np.argmax(np.cumsum(S[::-1] ** 2) / S[0] > eps)
        G[i] = G[i][:rank, :]
        G[i] = np.reshape(G[i], (-1, n, rank_tmp))
        U = U[:, :rank] * S[np.newaxis, :rank]
    G[0] = G[0] @ U
    return G

In [333]:
N = 32
A = np.zeros([N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            A[i,j,k] = np.sin(i+2.*j+3.*k)
TT_squeeze(TT_SVD(A, [10, 10, 10]))

[array([[-21.75679277,   6.35293037],
        [ -6.31769959,  21.64651922],
        [ 14.92985746,  17.03839812],
        [ 22.45097241,  -3.23474763],
        [  9.33076687, -20.53388133],
        [-12.3681027 , -18.95425923],
        [-22.69579569,   0.05182139],
        [-12.15707879,  19.01025767],
        [  9.55880029,  20.49075071],
        [ 22.48636246,   3.13214205],
        [ 14.74006669, -17.10614357],
        [ -6.55817842, -21.61711968],
        [-21.82686453,  -6.25341565],
        [-17.02803206,  14.85964989],
        [  3.42629457,  22.31082185],
        [ 20.73050176,   9.24952709],
        [ 18.97518124, -12.31574021],
        [ -0.2258334 , -22.55797277],
        [-19.21921786, -12.06050919],
        [-20.54254205,   9.52533092],
        [ -2.97914782,  22.35362571],
        [ 17.32326118,  14.63010011],
        [ 21.69874374,  -6.54427206],
        [  6.12450137, -21.70187068],
        [-15.08057931, -16.90686948],
        [-22.42064493,   3.43222955],
        [ -9

# ALS

In [266]:
def ALS(A, T, dimensions, rank):
    d = len(dimensions)
    for i in range(10000):
        for j in range(0, d):
            B = np.full((1, rank), fill_value=1)
            C = np.full((rank, rank), fill_value=1)
            for k in range(0, d):
                if (k != j):
                    C = np.multiply(C, A[k].T @ A[k])
                    B = linalg.khatri_rao(B, A[k])
            Y = B.T @ np.moveaxis(T, j, 0).reshape(T.shape[j], -1).T
            A[j] = (linalg.pinv(C) @ Y).T
    res = np.einsum('ir,jr,kr,wr', *A)
    err = np.linalg.norm(res - T) / np.linalg.norm(T)
    print(err)
    return A, err

In [267]:
N = 10
dim = (N, N, N, N)
rank = (1, 2, 4, 8, 16)
tensor = np.zeros([N,N,N,N])
for i in range(N):
    for j in range(N):
        for k in range(N):
            for q in range(N):
                tensor[i,j,k, q] = np.sin(i+2*j+3*k) + 4*q
for j in rank:
    A = [np.random.normal(size=(i, j)) for i in dim]
    A, err = ALS(A, tensor, dim, j)

0.03315319015555635
0.0236105164344558
2.1584386748628728e-14
0.00010190671076600212
6.069735406989148e-06
